In [ ]:
✅ SPLUNK DAILY HEALTH CHECK (ORDERED RUNBOOK)
🟦 1) Monitoring Console – Quick Overall Status (First 5 mins)

📍 Monitoring Console > Overview

✅ Check:

Any critical red alerts?

CPU / MEM spikes?

Indexing drop?

Search latency spike?

🟧 2) Indexer Health Check (IDX) ✅ (Most critical)

📍 Monitoring Console > Indexing > Performance
Check:

Indexing rate (EPS)

Throughput

CPU / Memory

📍 Resource Usage > Disk
Check:

Hot/Warm disk < 80%

Sudden disk growth

🔎 SPL:

index=_internal sourcetype=splunkd (ERROR OR WARN)


✅ Queue blocking:

index=_internal sourcetype=splunkd group=queue
| stats max(current_size) as max_size by name
| sort - max_size


🛠 T/S:

Disk high → alert infra + check retention/frozen

Indexing drop → check forwarders / pipeline

Queue blocked → CPU/disk I/O/noisy source

🟩 3) Data Ingestion / Data Gap Check ✅ (Daily must)

✅ Last 15 min ingestion:

index=* earliest=-15m
| stats count by index sourcetype
| sort -count


✅ Gap detection:

| tstats latest(_time) as lastTime where index=* by index sourcetype
| eval gap=now()-lastTime
| where gap>900
| eval gap_min=round(gap/60,2)
| table index sourcetype gap_min lastTime


🛠 T/S:

If gap → check forwarder service, inputs.conf, outputs.conf, connectivity

🟦 4) Search Head Health (SH)

📍 Monitoring Console > Search > Instance

Check:

Concurrency

Search latency

Long running searches

🔎 SPL:

index=_internal sourcetype=splunkd ERROR host=<search_head_host>


Find long running searches:

index=_internal sourcetype=splunkd "search finished"
| stats max(total_run_time) as maxTime by user savedsearch_name
| sort -maxTime


🛠 T/S:

Too many searches → optimize SPL / reduce time windows / tune schedule

🟪 5) Scheduler / Alert Health (ES / Reports)

📍 Monitoring Console > Scheduler Activity

Check:

skipped searches

delayed scheduled alerts

🔎 SPL:

index=_internal sourcetype=scheduler status=skipped OR status=failed
| stats count by savedsearch_name status
| sort -count


🛠 T/S:

reschedule overlaps

reduce load, tune correlation searches

🟨 6) Forwarders / Heavy Forwarder Health (HF/UF)

📍 HF UI → Settings > Forwarder Monitoring

🔎 SPL:

index=_internal sourcetype=splunkd component=TcpOutputProc (ERROR OR WARN)


🛠 T/S:

check port 9997 connectivity

restart forwarder service if allowed

🟥 7) License Manager Check (Daily Must)

📍 Settings > Licensing

🔎 SPL:

index=_internal source=*license_usage.log


Top consumers:

index=_internal source=*license_usage.log
| stats sum(b) as bytes by idx
| eval GB=round(bytes/1024/1024/1024,2)
| sort -GB


🛠 T/S:

identify noisy index/sourcetype

apply filtering / stop debug logs

🟫 8) Cluster Health Checks (If enabled)
✅ Indexer Cluster

📍 Indexer Clustering
Check:

replication/search factor

peer status

bucket fix issues

✅ Search Head Cluster (SHC)

Check captain + member sync:

$SPLUNK_HOME/bin/splunk show shcluster-status

🟪 9) Deployer / Deployment Server Checks
✅ SHC Deployer

Deployer errors:

index=_internal sourcetype=splunkd host=<deployer_host> (bundle OR deploy OR shcluster) (ERROR OR FAIL)

✅ Deployment Server

📍 Forwarder Management

forwarder last phone home

server class status

✅ 10) End of Shift Summary (Handover)

Send:

Tickets closed

Pending tickets + next action

Data gaps

Disk/license risks

Escalations